## Example from ML Flow website

In [50]:

import pandas as pd
import pathlib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

np.random.seed(20)

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def train(model_name, alpha, l1_ratio):
    
    data_path = pathlib.Path().absolute().parent / "data/wine-quality.csv"
    data = pd.read_csv(data_path)

    # Split the data into training and test sets.
    train, test = train_test_split(data, test_size=0.3)

    # The predicted column is "quality". 
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Useful for multiple runs (only doing one run in this sample notebook)  
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [51]:
train("model_1", 0.5, 0.2)

Elasticnet model (alpha=0.500000, l1_ratio=0.200000):
  RMSE: 0.8314332388357414
  MAE: 0.6363617450826506
  R2: 0.16045464844740842


In [52]:
train("model_2", 0.3, 0.2)

Elasticnet model (alpha=0.300000, l1_ratio=0.200000):
  RMSE: 0.8222252029987078
  MAE: 0.6333548541593412
  R2: 0.17894742870731317


In [53]:
train("model_3", 0.1, 0.2)

Elasticnet model (alpha=0.100000, l1_ratio=0.200000):
  RMSE: 0.8152628378714785
  MAE: 0.6313223823129606
  R2: 0.19279342873643146


In [1]:
from  mlflow.tracking import MlflowClient
client = MlflowClient()
experiments = client.list_experiments()

In [2]:
experiments

[<Experiment: artifact_location='/Users/federica/Documents/GitHub/ml_pipeline/model/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default'>]